In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
# from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
from sklearn import metrics
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
import nltk
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize
import numpy as np # linear algebra
import pandas as pd #data processing
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
import os
import re
import nltk

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Scrapingreviews.csv')

In [ ]:
tokenizer = nltk.RegexpTokenizer(r"\w+")

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pattern = '[0-9]'
real_words = []
# iterate through the csv file 
for val in df.reviews: 
    # split the value 
    tokens_words = tokenizer.tokenize(re.sub(pattern, '', str(val)))
    # Converts each token into lowercase 
    for i in range(len(tokens_words)): 
        tokens_words[i] = tokens_words[i].lower() 
    real_words.append(tokens_words)

In [ ]:
real_words

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words = list(stopwords.words('english'))

In [ ]:
# filtered_sentence = [w for w in real_words if not w.lower() in stop_words]
filtered_sentence = []
filtered_strgs=[]
for w in real_words:
    if w not in stop_words:
        filtered_sentence.append(w)
        filtered_strings=''
        for s in w:
          filtered_strings= filtered_strings + ' '+ s
        filtered_strgs.append(filtered_strings)

In [ ]:
len(filtered_strgs)

13642

In [ ]:
sentences=filtered_strgs.copy()

In [ ]:
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

dataframe=pd.DataFrame(columns=['sentence','compound','comp_score'],index=range(13642))
sia = SentimentIntensityAnalyzer()

for sentence in sentences:
    polarity = sia.polarity_scores(sentence)
    dataframe['sentence'].iloc[int(sentences.index(sentence))] = sentence
    dataframe['compound'].iloc[sentences.index(sentence)] = polarity['compound']
    dataframe['comp_score'].iloc[sentences.index(sentence)] = getAnalysis(dataframe['compound'].iloc[sentences.index(sentence)])

In [ ]:
dataframe

In [ ]:
dataframe.to_csv('/content/drive/MyDrive/Analysis_sentiment_Doc/analysis_sentiment.csv')

In [ ]:
from sklearn.cluster import AffinityPropagation

In [ ]:
dataframe.drop_duplicates(subset=['sentence'], keep = 'first',inplace= True)

In [ ]:
dataframe['comp_score'].value_counts()

In [ ]:
dataframe.reset_index(drop=True,inplace=True)

In [ ]:
dataframe['sentence'].isna().sum()

In [ ]:
dataframe.dropna(inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf_idf_vect = CountVectorizer(analyzer='word',ngram_range=(1,1),stop_words='english', min_df = 0.0001)
tf_matrix=tf_idf_vect.fit_transform(dataframe['sentence'])

In [ ]:
!pip install KMeans

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# implement kmeans
num_clusters = 3
km = KMeans(n_clusters=num_clusters)
km.fit(tf_matrix)
clusters = km.labels_.tolist()

In [ ]:
len(clusters)

9835

In [ ]:
dataframe.shape

(9835, 4)

In [ ]:
dataframe

,Unnamed: 0,sentence,compound,comp_score
0,0,i m not sure about data authentication but ni...,0.4971,Positive
1,1,display clearity is not as expected battery p...,0.7351,Positive
2,2,most functions except alexa are working as ex...,0.9346,Positive
3,3,boat xtend st review smart watch working very...,0.9619,Positive
4,4,very ordinary watch packing looks like a chea...,0.1280,Positive
...,...,...,...,...
9831,13637,best smartwatch in the market for android goo...,0.9042,Positive
9832,13638,most important features works only with samsu...,0.2716,Positive
9833,13639,i buy this watch two weeks ago and its help m...,0.9409,Positive
9834,13640,worst battery life but option are more so if ...,-0.3716,Negative


In [ ]:
# dataframe.drop(columns=['cluster'],inplace=True)

In [ ]:
dataframe.drop(columns=['second_cluster'],inplace=True)

In [ ]:
dataframe['kmeans-cluster']= pd.Series(clusters)

In [ ]:
dataframe['comp_score'].value_counts()

Positive    6717
Negative    2508
Neutral      610
Name: comp_score, dtype: int64

In [ ]:
dataframe.columns

Index(['Unnamed: 0', 'sentence', 'compound', 'comp_score', 'kmeans-cluster'], dtype='object')

In [ ]:
dataframe['kmeans-cluster'].value_counts()

1.0    8808
0.0    1022
2.0       4
Name: kmeans-cluster, dtype: int64

In [ ]:
af = AffinityPropagation(preference=-50)
af.fit(X=tf_matrix)
clusters1 = af.labels_.tolist()

/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_affinity_propagation.py:226: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  "will not have any cluster centers.", ConvergenceWarning)


In [ ]:
dataframe['Affinity-cluster']= pd.Series(clusters1)

In [ ]:
dataframe

,Unnamed: 0,sentence,compound,comp_score,kmeans-cluster,Affinity-cluster
0,0,i m not sure about data authentication but ni...,0.4971,Positive,2.0,-1.0
1,1,display clearity is not as expected battery p...,0.7351,Positive,2.0,-1.0
2,2,most functions except alexa are working as ex...,0.9346,Positive,2.0,-1.0
3,3,boat xtend st review smart watch working very...,0.9619,Positive,0.0,-1.0
4,4,very ordinary watch packing looks like a chea...,0.1280,Positive,2.0,-1.0
...,...,...,...,...,...,...
9831,13637,best smartwatch in the market for android goo...,0.9042,Positive,2.0,-1.0
9832,13638,most important features works only with samsu...,0.2716,Positive,2.0,-1.0
9833,13639,i buy this watch two weeks ago and its help m...,0.9409,Positive,2.0,-1.0
9834,13640,worst battery life but option are more so if ...,-0.3716,Negative,2.0,-1.0


In [ ]:
dataframe[dataframe['kmeans-cluster'] == 1.0]

,Unnamed: 0,sentence,compound,comp_score,kmeans-cluster,Affinity-cluster
0,0,i m not sure about data authentication but ni...,0.4971,Positive,1.0,-1.0
1,1,display clearity is not as expected battery p...,0.7351,Positive,1.0,-1.0
2,2,most functions except alexa are working as ex...,0.9346,Positive,1.0,-1.0
4,4,very ordinary watch packing looks like a chea...,0.1280,Positive,1.0,-1.0
5,5,have been using it for day now and i have to ...,0.7753,Positive,1.0,-1.0
...,...,...,...,...,...,...
9830,13636,not worth for money look wise is very bad sum...,-0.4978,Negative,1.0,-1.0
9831,13637,best smartwatch in the market for android goo...,0.9042,Positive,1.0,-1.0
9832,13638,most important features works only with samsu...,0.2716,Positive,1.0,-1.0
9833,13639,i buy this watch two weeks ago and its help m...,0.9409,Positive,1.0,-1.0


In [ ]:
dataframe[dataframe['kmeans-cluster'] == 2.0]

,Unnamed: 0,sentence,compound,comp_score,kmeans-cluster,Affinity-cluster
579,643,i ve been using mi band and this one obviousl...,-0.0495,Negative,2.0,-1.0
2565,3214,i have been using a mi band already so this i...,0.9975,Positive,2.0,-1.0
2664,3325,over all the new mi band is a cost effective ...,0.9608,Positive,2.0,-1.0
3048,3710,mi band is a killer in its price range overal...,0.9381,Positive,2.0,-1.0


In [ ]:
dataframe[dataframe['kmeans-cluster'] == 0.0]

,Unnamed: 0,sentence,compound,comp_score,kmeans-cluster,Affinity-cluster
3,3,boat xtend st review smart watch working very...,0.9619,Positive,0.0,-1.0
7,7,your browser does not support html video my s...,0.9416,Positive,0.0,-1.0
13,22,watch is very beautiful and it reaches the cu...,0.8070,Positive,0.0,-1.0
14,23,i brought so many products of boat brand and ...,0.9826,Positive,0.0,-1.0
15,24,i would say its a best watch under few key po...,0.9888,Positive,0.0,-1.0
...,...,...,...,...,...,...
9714,13511,this is the best that your android can work w...,0.8020,Positive,0.0,-1.0
9723,13520,awesome watch but i preferred a rubber strap ...,0.9169,Positive,0.0,-1.0
9815,13621,my friend bought this watch and i bought a sm...,-0.0429,Negative,0.0,-1.0
9816,13622,i have watch lte model mm it is pretty expens...,0.9769,Positive,0.0,-1.0


In [ ]:
from sklearn.cluster import AgglomerativeClustering
aglo = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='single')
dataframe['Aglo-cluster']=aglo.fit_predict(tf_matrix.toarray())

In [ ]:
dataframe[dataframe['kmeans-cluster'] == 2.0]

,Unnamed: 0,sentence,compound,comp_score,kmeans-cluster,Affinity-cluster,Aglo-cluster
579,643,i ve been using mi band and this one obviousl...,-0.0495,Negative,2.0,-1.0,0
2565,3214,i have been using a mi band already so this i...,0.9975,Positive,2.0,-1.0,0
2664,3325,over all the new mi band is a cost effective ...,0.9608,Positive,2.0,-1.0,0
3048,3710,mi band is a killer in its price range overal...,0.9381,Positive,2.0,-1.0,0


In [ ]:
# birch clustering
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import Birch
from matplotlib import pyplot
X = tf_matrix
model = Birch(threshold=0.01, n_clusters=3)
# fit the model
model.fit(X)
# assign a cluster to each example
y = model.predict(X)
# retrieve unique clusters
clusters = unique(y)

In [ ]:
dataframe['Birch_cluster']= clusters